In [2]:
import math

import numpy as np
from scipy.stats import f
from scipy.stats import norm, laplace

## 1.3
$X_1 ... X_n \sim N(\mu_1, \sigma_1^2)$\
$Y_1 ... Y_m \sim N(\mu_2, \sigma_2^2)$\
$X, Y - $ независимы

$\tau = \frac{\sigma_1^2}{\sigma_2^2}, \mu_1, \mu_2$ - незвестны

По теореме Фишера:

$U_1 = \frac{n S_{1*}^2}{\sigma_1^2} \sim \chi (n-1)$\
$U_2 = \frac{n S_{2*}^2}{\sigma_2^2} \sim \chi (m-1)$

Воспользуемся распределением Фишера

$\frac{\frac{U_2}{m-1}}{\frac{U_1}{n-1}} \sim F(n, m)$\
$\frac{m(n-1)S_{2*}^2 \sigma_1^2}{n(m-1)S_{1*}^2 \sigma_2^2} \sim F(n, m)$\
$P(q_{\frac{\alpha}{2}} <= \frac{m(n-1)S_{2*}^2 \sigma_1^2}{n(m-1)S_{1*}^2 \sigma_2^2} <= q_{1 - \frac{\alpha}{2}})$
$P(\frac{\sigma_1^2}{\sigma_2^2} in [q_{\frac{\alpha}{2}} \frac{n(m-1)S_{1*}^2}{m(n-1)S_{2*}^2}; q_{1 - \frac{\alpha}{2}}\frac{n(m-1)S_{1*}^2}{m(n-1)S_{2*}^2}])$


In [3]:
mu1 = 0
mu2 = 0
sigma1=2
sigma2=1
alpha = 0.05
n_s = [25, 10000]
for it in n_s:
    cnt = 0
    n = m = it
    F_lower = f.ppf(alpha/2, n-1, m-1)
    F_upper = f.ppf(1 - alpha/2, n-1, m-1)
    for i in range(1000):
        X = norm.rvs(loc=mu2, scale=math.sqrt(sigma1),size=n)
        Y = norm.rvs(loc=mu2, scale=sigma2, size=m)
        var_X = np.var(X)
        var_Y = np.var(Y)
        q = n*(m-1) * var_X / (m*(n-1)*var_Y)
        if F_upper * q >= 2 >= F_lower * q:
            cnt += 1
    print("Stats: ", cnt, "from ", 1000)


Stats:  956 from  1000
Stats:  949 from  1000


Результаты эксперемента показывают, что доверительный интервал найден верно. Попаданий больше, чем 95%.

Процент попадания осталься примерно таким же.

## 2.2

Пусть $X_1, \dots, X_n \sim \mathrm{Laplace}(\mu, \alpha)$,  
где $\mu$ — параметр сдвига (неизвестен), а масштаб $\alpha$ известен.



### Из теоремы о средних членах вариационного ряда:
$\sqrt n f(q_{p}) \dfrac {X_{(n p)} - q_{p}} {\sqrt {p (1 - p)}} \xrightarrow{d} N(0, 1)$

где $f$ — плотность распределения Лапласа

Тогда, взяв $p = 1/2$, из двойного неравенства, найдём асимптотический доверительный интервал:
$$
P\!\big(q_{p/2} \le 2 \sqrt n f(\mu) (X_{(n / 2)} - \mu) \le q_{1-p/2}\big) = 1 - p,
$$
где $q_p$ — $p$-й квантиль распределения $N(0, 1)$.

Так как  
$$2\sqrt n\, f(\mu) = \sqrt n \alpha,$$
получаем:

$$X_{(n/2)} - \frac {1}{\alpha {\sqrt n}} q_{1-p/2} \le \mu \le X_{(n/2)} - \frac {1} {\alpha {\sqrt n}} q_{p/2}.$$




In [11]:
mu = 2
scale = 1
alpha = 0.05
n_s = [25, 10000]
for it in n_s:
    cnt = 0
    n = it
    left_quantile = norm.ppf(alpha/2)
    right_quantile = norm.ppf(1 - alpha/2)
    for i in range(1000):
        X = laplace.rvs(loc=mu, scale=scale, size=n)
        med_X = np.median(X)
        left = med_X - right_quantile / np.sqrt(it) / scale
        right = med_X - left_quantile / np.sqrt(it)/ scale
        if left <= mu <= right:
            cnt += 1
    print("Stats: ", cnt, "from ", 1000)


Stats:  901 from  1000
Stats:  951 from  1000
